# Aga Patro - lab 2

## Zadanie 1. Implementacja algorytmów 

### 1.1 Statyczny algorytm Huffmana

In [5]:
from heapq import heappop, heappush, heapify
from bitarray import bitarray
from bitarray.util import ba2int

In [6]:
class Node:
    def __init__(self, value, letter=None, left=None, right=None):
        self.value = value
        self.letter = letter
        self.left = left
        self.right = right

    def __gt__(self, other):
        return self.value > other.value


def count_letters(text):
    letters = {}
    for letter in text:
        letters[letter] = letters.get(letter, 0) + 1

    return letters


def build_static_huffman_tree(text):
    counted_letters = count_letters(text)
    leafs = [Node(weight, letter) for letter, weight in counted_letters.items()]
    while len(leafs) > 1:
        el_1 = heappop(leafs)
        el_2 = heappop(leafs)
        heappush(leafs, Node(el_1.value + el_2.value, left=el_1, right=el_2))

    return leafs[0]


def create_huffman_code(node, codes, code):
    if node.letter is not None:
        codes[node.letter] = code

    code_cpy = code.copy()
    if node.left is not None:
        code.append(0)
        create_huffman_code(node.left, codes, code)

    if node.right is not None:
        code = code_cpy
        code.append(1)
        create_huffman_code(node.right, codes, code)

    return codes


def encode_static_huffman_tree(text, codes):
    result = bitarray()
    for letter in text:
        result.extend(codes[letter])

    return result


def decode_static_huffman_tree(huffman_tree, encoded_text):
    node = huffman_tree
    decoded_text = ''
    for bit in encoded_text:
        if not bit:
            node = node.left
        else:
            node = node.right

        if not node.left and not node.right:
            decoded_text += node.letter
            node = huffman_tree

    return decoded_text


def static_huffman(text):
    huffman_tree = build_static_huffman_tree(text)
    codes = create_huffman_code(huffman_tree, dict(), bitarray())
    encoded_text = encode_static_huffman_tree(text, codes)
    decoded_text = decode_static_huffman_tree(huffman_tree, encoded_text)

    return decoded_text

In [7]:
text = "Let's check if it work"
decoded_text = static_huffman(text)
print(f"Text: {text}\nDecoded text: {decoded_text}")

Text: Let's check if it work
Decoded text: Let's check if it work


## Zadanie 2. Pomiary kompresji oraz dekompresji

### 2.1 Format pliku
Wynik kompresji zapisywany jest w postaci bitowej przy użyciu "bitarray", co powinno rozwiązać problem konwersji na znaki ASCII oraz niepełnych bitów na końcu pliku. Plik wynikowy jest w postaci binarnej.

### 2.3 Pomiar wpółczynnika oraz czasu kompresji 

#### Pliki wejściowe. 
Do pomiarów wybrałam następujące pliki:
* "History of Astronomy" z projektu Gutenberg
* plik źródłowy "crypto_engine.c" jądra Linuksa
* stworzyłam swój własny plik ze znakami losowymi

Każdy z plików przerobiłam tak, by miały wymagane rozmiary

In [8]:
import os
import time


def compression_ratio(read_file, write_file):
    read_size = os.path.getsize(read_file)
    print("Size before compression:", read_size)
    write_size = os.path.getsize(write_file)/8
    print("Size after compression: ", write_size) 
    print(f"Size ratio: {1 - write_size/read_size}") 


def compression_test(filename):
    with open(filename, "r") as file:
        text = file.read()
    
    print("\n--------------------------")
    print("Static Huffman compression")
    print("--------------------------\n")
    result_filename = filename.replace('input', 'output')
    with open(result_filename, "wb+") as file:
        file.truncate()

        static_tree = build_static_huffman_tree(text)
        codes = create_huffman_code(static_tree, dict(), bitarray())

        start_e = time.time()
        encoded_text = encode_static_huffman_tree(text, codes)
        end_e = time.time()

        encoded_text.tofile(file)

        start_d = time.time()
        decoded_text = decode_static_huffman_tree(static_tree, encoded_text)
        end_d = time.time()

        file.close()
    

    compression_ratio(filename, result_filename)
    time_e1 = end_e - start_e
    print("Time encoding:", time_e1)
    time_d1 = end_d - start_d
    print("Time decoding", time_d1)
    

In [9]:
sizes = ['1kb', '10kb', '100kb', '1mb']

for size in sizes:
    compression_test(f'lab_2/input/hoa_{size}.txt')


--------------------------
Static Huffman compression
--------------------------

Size before compression: 1024
Size after compression:  84.375
Size ratio: 0.9176025390625
Time encoding: 0.00010204315185546875
Time decoding 0.0003960132598876953

--------------------------
Static Huffman compression
--------------------------

Size before compression: 10318
Size after compression:  788.25
Size ratio: 0.9236043806939329
Time encoding: 0.0007097721099853516
Time decoding 0.0036611557006835938

--------------------------
Static Huffman compression
--------------------------

Size before compression: 102942
Size after compression:  7405.25
Size ratio: 0.9280638612033961
Time encoding: 0.006725311279296875
Time decoding 0.03512692451477051

--------------------------
Static Huffman compression
--------------------------

Size before compression: 1054067
Size after compression:  77243.875
Size ratio: 0.9267182494091931
Time encoding: 0.06788754463195801
Time decoding 0.3600809574127197


In [10]:
sizes = ['1kb', '10kb', '100kb', '1mb']

for size in sizes:
    compression_test(f'lab_2/input/crypto_engine_{size}.c')


--------------------------
Static Huffman compression
--------------------------

Size before compression: 1024
Size after compression:  83.0
Size ratio: 0.9189453125
Time encoding: 9.322166442871094e-05
Time decoding 0.0003845691680908203

--------------------------
Static Huffman compression
--------------------------

Size before compression: 10240
Size after compression:  803.25
Size ratio: 0.9215576171875
Time encoding: 0.0008006095886230469
Time decoding 0.004292964935302734

--------------------------
Static Huffman compression
--------------------------

Size before compression: 102400
Size after compression:  8179.375
Size ratio: 0.920123291015625
Time encoding: 0.00765681266784668
Time decoding 0.03764486312866211

--------------------------
Static Huffman compression
--------------------------

Size before compression: 1048576
Size after compression:  83278.25
Size ratio: 0.9205796718597412
Time encoding: 0.07568526268005371
Time decoding 0.38315606117248535


In [11]:
sizes = ['1kb', '10kb', '100kb', '1mb']

for size in sizes:
    compression_test(f'lab_2/input/random_data_{size}.txt')


--------------------------
Static Huffman compression
--------------------------

Size before compression: 1024
Size after compression:  106.5
Size ratio: 0.89599609375
Time encoding: 0.00010347366333007812
Time decoding 0.00047397613525390625

--------------------------
Static Huffman compression
--------------------------

Size before compression: 10240
Size after compression:  1068.125
Size ratio: 0.89569091796875
Time encoding: 0.0009298324584960938
Time decoding 0.004800319671630859

--------------------------
Static Huffman compression
--------------------------

Size before compression: 102400
Size after compression:  10704.5
Size ratio: 0.8954638671875
Time encoding: 0.009365558624267578
Time decoding 0.04767870903015137

--------------------------
Static Huffman compression
--------------------------

Size before compression: 1048576
Size after compression:  109718.625
Size ratio: 0.8953641653060913
Time encoding: 0.09615325927734375
Time decoding 0.4834096431732178
